In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import torch


c:\Users\4pygm\miniconda3\envs\breast\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
x = [torch.rand(size=(24, 24)) for _ in range(4)]

In [15]:
import numpy as np
import torch.utils.data as data_utils
from torchvision import datasets, transforms
loader = data_utils.DataLoader(datasets.MNIST('../datasets',
                    train=True,
                    download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))])),
    batch_size=100,
    shuffle=False)

In [16]:
 for (batch_data, batch_labels) in loader:
    all_imgs = batch_data
    all_labels = batch_labels


In [9]:
np.random.RandomState(32).randint(0, 10, 5)

array([7, 5, 6, 8, 3])

In [6]:
torch.autograd.Variable(x)

TypeError: Variable data has to be a tensor, but got list

In [ ]:
import os

ROOT_DIR = os.getcwd()

create_bags()